In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.database import database
from utils.ethogram import ethogram

Autosaving every 180 seconds


In [2]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'

cohort = database.CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

# print (list(zip(cohort.PDays,cohort.Start_times))[0])
# print (list(zip(cohort.PDays,cohort.Start_times))[1])
# print (list(zip(cohort.PDays,cohort.Start_times))[100])

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0         0.0     2.0     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_09_53_59_813912_compressed_Day.slp   09:53:59
1         1.0     2.0     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_10_16_08_629297_compressed_Day.slp   10:16:08
2         2.0     2.0     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_10_38_20_043555_compressed_Day.slp   10:38:20
3         3.0     2.0     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_11_00_31_566455_compressed_Day.slp   11:00:31
4         4.0     2.

In [3]:
######################################################
############# PREPROCESS FEATURE TRACKS ##############
######################################################
cohort.n_cores = 30
cohort.parallel = False
cohort.interpolate_flag = False
cohort.preprocess_feature_tracks()

#

100%|██████████████████████████████████████| 939/939 [00:00<00:00, 63985.76it/s]


In [4]:
#########################################################
############## PREPROCESS HUDDLES #######################
#########################################################

# 
cohort.n_cores = 30
cohort.parallel = True
cohort.fix_track_flag = True
cohort.interpolate_flag = True
cohort.overwrite_flag = False
cohort.no_huddle_features = True
cohort.preprocess_huddle_tracks()

#

100%|███████████████████████████████████████| 939/939 [00:00<00:00, 4004.11it/s]


In [ ]:
#######################################################
########### REMOVE HUDDLES FROM FEATURE TRACKS ########
#######################################################

#
cohort.huddle_min_distance = 200    # min distance from huddle in pixels for feature to be kept

# this uses the _spine.npy data for tracks; and _huddle_spine_fixed_interpolated.npy for huddles
# if we want to use something else, we can also code it
cohort.parallel = True
cohort.n_cores = 30
cohort.remove_huddles_from_feature_tracks()

#


# file pairs found:  939  (if less  than above, please check missing)


 69%|████████████████████████████▎            | 648/939 [01:34<00:31,  9.29it/s]

In [5]:
######################################################
########## TIME IN PROXIMITY BEHAVIOR ################
######################################################

#
cohort.track_type = 'features'
cohort.no_huddle_features = True

n_partitions = 24
behavior_name = 'pairwise_proximity'

# select animals
animal_ids = [0,1,2,3,4,5]

all_pairs = [(a, b) for idx, a in enumerate(animal_ids) for b in animal_ids[idx + 1:]]
print ("all paris: ", all_pairs)

#
for pair in all_pairs:

    cohort.animal_ids = [pair[0],pair[1]]
    
    #
    cohort.get_pairwise_interaction_time(cohort.animal_ids[0], cohort.animal_ids[1])

    #
    cohort.format_behavior()

    #
    ethogram.generate_ethogram_hourly(n_partitions, 
                                      behavior_name,
                                      cohort,
                                      cohort.no_huddle_features,
                                      vmax=50)
    # for plotting use code below


all paris:  [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]


 56%|█████▌    | 838/1500 [00:01<00:01, 455.75it/s] 


KeyboardInterrupt: 

In [12]:
################################################
########### CREATE ROI OF INTEREST #############
################################################
# need to select 1 video and 1 slp for visualization purposes;
# make sure the video is shirnk cropped and matches the traces from the slop also.
cohort.fname_video ='/media/cat/4TBSSD/dan/cohort2/P21_P28/2020_07_26_00_05_52_835502_compressed_defished_shrink_cropped.mp4'
cohort.fname_slp_npy =  '/media/cat/4TBSSD/dan/cohort2/P21_P28/features/2020_07_26_00_05_52_835502_compressed_Night_spine.npy'

#
cohort.load_video()

#
cohort.set_roi()

print ("rectangular coords: ", cohort.rect_coords)
print ("circular coords: ", cohort.circle_coords)


(28802, 6, 2)
rectangular coords:  [[328.68114144 267.27272727]
 [528.78039702 427.27272727]]
circular coords:  [[159.84739454 490.        ]
 [236.67121588 548.18181818]]


In [13]:
################################################
########### COMPUTE CIRCLE OCCUPANCY ###########
################################################

#
behavior_name = 'circular_region'

# select animals
cohort.no_huddle_features = True

# select animals
animal_ids = [0,1,2,3,4,5]

#
for animal_id in animal_ids:

    #
    #cohort.animals = animal_id
    cohort.animal_ids = animal_id

    #
    cohort.get_circle_occupancy(cohort.animal_ids)

    #
    cohort.format_behavior()
    
    #
    ethogram.generate_ethogram_hourly(n_partitions, 
                                      behavior_name,
                                      cohort,
                                      cohort.no_huddle_features,
                                      vmax=50)

100%|██████████| 1500/1500 [00:00<00:00, 4611.45it/s]


In [17]:
################################################
########### COMPUTE RECTANGLE OCCUPANCY ########
################################################

#
behavior_name = 'rectangular_region'

# select animals
cohort.no_huddle_features = True

# select animals
animal_ids = [0,1,2,3,4,5]

#
for animal_id in animal_ids:

    #
    cohort.animal_ids = animal_id

    #
    cohort.get_circle_occupancy(cohort.animal_ids)

    #
    cohort.format_behavior()
    
    #
    ethogram.generate_ethogram_hourly(n_partitions, 
                                      behavior_name,
                                      cohort,
                                      cohort.no_huddle_features,
                                      vmax=50)

100%|██████████| 1500/1500 [00:00<00:00, 4622.69it/s]


In [69]:
######################################
########### PCA SCATTER PLOTS ########
######################################

#
animal_ids = [0,1,2,3,4,5]

# choose one of these behaviors
cohort.behavior_name = 'rectangular_region'
cohort.behavior_name = 'circular_region'
cohort.behavior_name = 'pairwise_proximity'

# for single animal behaviors
cohort.animal_ids = animal_ids

# for animal-pair interactions
cohort.animal_ids = list((a, b) for idx, a in enumerate(animal_ids) for b in animal_ids[idx + 1:])

#
cohort.remove_zeros=True
cohort.show_developmental_trajectories()

#


[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]
(240, 24)
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.38157072e-02 3.47198111e-03 1.94430942e-01 ... 6.33983751e+00
  1.68391084e-01 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
X_pca:  (240, 3)
(240,)


In [16]:
#######################################################
########### REMOVE HUDDLES FROM FEATURE TRACKS ########
#######################################################

#
track1 = np.load('/media/cat/4TBSSD/dan/cohort2/P21_P28/features/2020_07_26_00_05_52_835502_compressed_Night_spine.npy')
track2 = np.load('/media/cat/4TBSSD/dan/cohort2/P21_P28/features/2020_07_26_00_05_52_835502_compressed_Night_spine_nohuddle.npy')

#
huddle = np.load('/media/cat/4TBSSD/dan/cohort2/P21_P28/huddles/2020_07_26_00_05_52_835502_compressed_Night_huddle_spine_fixed_interpolated.npy')
print (huddle.shape)

print (track1.shape)
animal_id = 0

# 
plt.figure()
# plt.plot(track1[:,animal_id,0],
#         track1[:,animal_id,1],
#         c='red',
#         alpha=.5)
# 
plt.plot(track2[:,animal_id,0],
        track2[:,animal_id,1]+10,
        c='blue',
        alpha=.5)
# 
plt.scatter(huddle[:,0,0], huddle[:,0,1],
           s=50,
           alpha=.5)
plt.xlim(0,900)
plt.ylim(0,700)
plt.show()

# 




(28802, 1, 2)
(28802, 6, 2)
